# Fault detection (inference)

Now let's the whole cube

In [ ]:
import os
import sys
import re
from copy import copy
import h5py

import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm_notebook
import scipy

sys.path.append('../../..')

from seismiqb import *

from seismiqb.batchflow import FilesIndex, Pipeline
from seismiqb.batchflow.research import Results
from seismiqb.batchflow.models.torch import EncoderDecoder, ResBlock, TorchModel
from seismiqb.batchflow import D, B, V, P, R, L, W

In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=7

In [ ]:
model_path = './research_1d_without_transpose/results/crop_(1, 128, 192)-paths_01_ETP-repetition_0-update_0/2150108386/model_2000'
inference_path = '/data/seismic_data/seismic_interpretation/CUBE_16_PSDM/PREDICTIONS/FAULTS/2150108386.hdf5'
cube = '/data/seismic_data/seismic_interpretation/CUBE_16_PSDM/amplitudes_16_PSDM'
cube_path = cube + '.hdf5'

In [ ]:
CROP_SHAPE = (1, 128, 192)
BATCH_SIZE = 40

In [ ]:
# OVERLAP_SHAPE = tuple(np.maximum((np.array(CROP_SHAPE) * 3/4).astype(int), np.array([1,1,1])))
STRIDE = (1, 64, 96)

In [ ]:
dataset = SeismicCubeset(FilesIndex(path=[cube_path], no_ext=True))
dataset.load(label_dir={
    'amplitudes_01_ETP': '/INPUTS/FAULTS/NPY/*',
    'amplitudes_16_PSDM': '/INPUTS/FAULTS/NPY/*',
}, labels_class=Fault, transform=True, verify=True)
geometry = dataset.geometries[0]

# Predict on WHOLE

In [ ]:
geometry = dataset.geometries[0]
path_hdf5 = inference_path

# ! rm -r {path_hdf5}

# file_hdf5 = h5py.File(path_hdf5, "a")
# cube_hdf5 = file_hdf5.create_dataset('cube', geometry.cube_shape)

In [ ]:
# inference_template = (
#     Pipeline()
#     # Initialize everything
#     .init_variable('result_preds', [])
#     .load_model(mode='dynamic', model_class=TorchModel, name='model', path=model_path)
#     # Load data
#     .crop(points=D('grid_gen')(),
#           shape=CROP_SHAPE)
#     .load_cubes(dst='images')
#     .adaptive_reshape(src='images', shape=CROP_SHAPE)
#     .scale(mode='q', src='images')

#     # Predict with model, then aggregate
#     .predict_model('model',
#                    B('images'),
#                    fetches='sigmoid',
#                    save_to=V('result_preds', mode='e'))
# )

In [ ]:
# def split_cube(geometry, crop_shape, stride, length):
#     il_min = 0
#     ranges = []
#     for il_max in range(length, geometry.ilines_len + stride, stride):
#         if il_max >= geometry.ilines_len:
#             il_max = geometry.ilines_len - 1
#         if il_max - il_min >= crop_shape[0]:
#             ranges += [[il_min, il_max]]
#         else:
#             ranges[-1][-1] = geometry.ilines_len-1
#         il_min += stride
#     return ranges

In [ ]:
# for il_min, il_max in split_cube(geometry, CROP_SHAPE, 80, 140):
#     print('bounds', il_min, il_max)
#     dataset.make_grid(dataset.indices[0], CROP_SHAPE,
#                       [il_min, il_max], [0, geometry.xlines_len-1], [0, geometry.depth-1],
#                       strides=STRIDE,
#                       batch_size=8) 

#     inference_pipeline = inference_template << dataset
#     for _ in tqdm(range(dataset.grid_iters)):
#         batch = inference_pipeline.next_batch(D('size'))

#     # Write to hdf5
#     slices = tuple([slice(*item) for item in dataset.grid_info['range']])
#     prediction = (dataset.assemble_crops(inference_pipeline.v('result_preds'), order=(0, 1, 2)) > 0.5).astype(int)
#     if il_min == 0:
#         cube_hdf5[:100, slices[1], slices[2]] = prediction[0:100]
#     elif il_max == geometry.ilines_len - 1:
#         cube_hdf5[il_min+20:-1, slices[1], slices[2]] = prediction[20:]
#     else:
#         cube_hdf5[il_min+20:il_max-20, slices[1], slices[2]] = prediction[20:120]
        
    
# file_hdf5.close()

In [ ]:
geometry_sgy = SeismicGeometry(path_hdf5)

In [ ]:
from numba import njit, prange

@njit(parallel=True)
def filter_array(array, result, window):
    for i in prange(0, array.shape[0]-window[0]+1):
        for j in prange(0, array.shape[1]-window[1]+1):
            for k in prange(0, array.shape[2]-window[2]+1):
                region = array[i:i+window[0], j:j+window[1], k:k+window[2]]
                denum = np.sum(region**2) * region.shape[0] * region.shape[1]
                if denum != 0:
                    result[i, j, k] = ((np.sum(np.sum(region, axis=0), axis=0)**2).sum()) / denum
    return result

def semblance(geometry, labels=None, locations=None, window=10):
    if isinstance(window, int):
        window = np.ones(3, dtype=np.int32) * window
    if labels:
        _min = geometry.cube_shape
        _max = np.zeros(3)
        for label in labels:
            _min = np.minimum(label.points.min(axis=0)+1, _min)
            _max =  np.maximum(label.points.max(axis=0)+1, _max)
    else:
        if locations:
            _min, _max = locations
            _min = np.array(_min)
            _max = np.array(_max)
        else:
            _min = np.zeros(3)
            _max = geometry.cube_shape
        
    _min = _min.astype(int)
    _max = _max.astype(int)

    cube = geometry.file_hdf5['cube'][_min[0]:_max[0], _min[1]:_max[1], _min[2]:_max[2]]
    window = np.minimum(np.array(window), cube.shape)

    def compute_marfurt_semblance(region):
        return (np.sum(region, axis=(0, 1))**2).sum() / (np.sum(region**2, axis=(0, 1)).sum() * len(region))

    return scipy.ndimage.generic_filter(cube,
                                        compute_marfurt_semblance,
                                        window)
    result = np.zeros_like(cube)
    return filter_array(cube, result, window)

In [ ]:
loc = 743# dataset.labels[0][1].points[1,0]

locations = geometry.make_slide_locations(loc, axis=0)
shape = np.array([(slc.stop - slc.start) for slc in locations])
seismic_slide = geometry.load_slide(loc)

mask = np.zeros_like(seismic_slide.reshape(shape))
for label in dataset.labels[0]:
    mask = label.add_to_mask(mask, locations)
    
prediction = geometry_sgy.load_slide(loc)

In [ ]:
%%time
sem = semblance(geometry, locations=[[locations[i].start for i in range(3)], [locations[i].stop for i in range(3)]])

In [ ]:
def iou(target, prediction):
    target = 1 - (np.nan_to_num(target, 0))
    return (prediction * target).sum() / prediction.sum()

In [ ]:
# semblance = scipy.ndimage.generic_filter(seismic_slide,
#                                          compute_marfurt_semblance,
#                                          window)

In [ ]:
sem.shape, prediction.shape

In [ ]:
iou(sem, prediction), iou(sem, mask[0])

In [ ]:
# prediction[semblance > 0.9] = 0

zoom_slice = (slice(0, 1), slice(None, None), slice(1000, 1500))
plot_image((sem[zoom_slice]), figsize=(20, 20), cmap='gray', show=True, colorbar=True)
plot_image((sem[zoom_slice], mask[zoom_slice], prediction[zoom_slice[1:]]), figsize=(20, 20), mode='overlap', color=('red', 'blue'), show=True) 

In [ ]:
# for slide in range(800):
#     mask = geometry_sgy.load_slide(slide)
#     if mask.sum() > 0:
#         print(slide, mask.sum())

slide = loc# dataset.labels[0][1].points[0, 0]

zoom_slice = (slice(None, None), slice(1000, 1500))


dataset.show_slide(slide, figsize=(20, 20), width=1, zoom_slice=zoom_slice)

mask = geometry_sgy.load_slide(slide)
slide = dataset.geometries[0].load_slide(slide)
plot_image((slide[:, zoom_slice[1]], mask[:, zoom_slice[1]]), mode='overlap', figsize=(20, 20), zoom_slice=zoom_slice)

In [ ]:
mask.sum()